In [35]:
import pandas as pd
import numpy as np
from scipy import signal, stats
from scipy.special import kl_div, rel_entr
from scipy.stats import ks_2samp
import warnings
warnings.filterwarnings('ignore')


## Audios Camilo

In [75]:
raw = pd.read_excel('../results/camilo_indices_v2.xlsx', engine='openpyxl')
pro = pd.read_excel('../results/camilo_indices_processed_v2.xlsx', engine='openpyxl')


raw.drop(columns=raw.columns[0], inplace=True)
pro.drop(columns=pro.columns[0], inplace=True)

In [76]:
sitios = ['G60','G21','S1','S2']
indices = ['ACIft','ADI','AEI','M','H','NP','BETA','NDSI'] 

## Fuciones

In [8]:

def get_pdf(data1,data2,number,xlim=None):
    kernel1 = stats.gaussian_kde(data1)
    kernel2 = stats.gaussian_kde(data2)
    if xlim == None:
        x = np.linspace(np.min(np.hstack((data1,data2))) - np.std(np.hstack((data1,data2))), 
            np.max(np.hstack((data1,data2)))+ np.std(np.hstack((data1,data2))), number )
    else:
        x = np.linspace(xlim[0], xlim[1], number )
    pdf1 = kernel1.pdf(x)
    pdf1 /= np.sum(pdf1)
    pdf2 = kernel2.pdf(x)
    pdf2 /= np.sum(pdf2)
    return pdf1,pdf2, x

def kl_divergence(p, q):
    n=0.01
    #return (np.sum(rel_entr(p,q)) + np.sum(rel_entr(q,p))) / 2
    return (np.sum(np.where(p >= n, p * np.log(p / q), 0)) + np.sum(np.where(q >= n, q * np.log(q / p), 0)))/2

def get_metrics(name, v1,v2,p,q):
    corr = np.round(np.corrcoef(v1,v2)[0,1],2)
    kld  = np.round(kl_divergence(p, q) + kl_divergence(q, p),2)/2
    _, p_value = stats.mannwhitneyu(p,q)
    p_value = np.round(1 - p_value,5)
    return [name,corr,kld,p_value]


def d_kolmogorov(p,q):
    pac1 = np.cumsum(p)
    pac2 = np.cumsum(q)

    dabs = np.abs(pac1 - pac2)

    return np.max(dabs)

In [88]:



for s in sitios:
    raw_a = raw.query(f"folder == '{s}A'")
    raw_s = raw.query(f"folder == '{s}S'")

    pro_a = pro.query(f"folder == '{s}A'")
    pro_s = pro.query(f"folder == '{s}S'")
    
    kld_r = []
    pvalue_r = []
    corr_r = []

    kld_p = []
    pvalue_p = []
    corr_p = []


    for y in indices:

        pdf1_r, pdf2_r, p = get_pdf(raw_a[y].values,raw_s[y].values,100)
        mr1 = kl_divergence(pdf1_r,pdf2_r)

        mr2 = np.round(np.corrcoef(raw_a[y].values,raw_s[y].values)[0,1],2)
        mr3 = d_kolmogorov(pdf1_r,pdf2_r)

        kld_r.append(np.round(mr1,3))
        corr_r.append(mr2)
        pvalue_r.append(np.round(mr3,3))



        pdf1_p, pdf2_p, x = get_pdf(pro_a[y].values,pro_s[y].values,100)
        mp1 = kl_divergence(pdf1_p,pdf2_p)
        mp2 = np.round(np.corrcoef(pro_a[y].values,pro_s[y].values)[0,1],2)
        mp3 = d_kolmogorov(pdf1_p,pdf2_p)

        kld_p.append(np.round(mp1,3))
        corr_p.append(mp2)
        pvalue_p.append(np.round(mp3,3))

    
    print("\n")
    print("#"*100)
    print(f"Site {s}")
    print("#"*100)
    print("raw kld"," & ".join(str(v) for v in kld_r) )
    print("pro kld"," & ".join(str(v) for v in kld_p) )
    print("\n")
    print("raw pvalue"," & ".join(str(v) for v in pvalue_r) )
    print("pro pvalue"," & ".join(str(v) for v in pvalue_p) )
    print("#"*100)
        






####################################################################################################
Site G60
####################################################################################################
raw kld 48.292 & inf & 4.831 & 3.497 & 0.913 & 0.188 & 0.138 & 0.011
pro kld 0.077 & 0.943 & 0.886 & 0.026 & 0.08 & 0.017 & 0.084 & 0.011


raw pvalue 0.991 & 0.926 & 0.855 & 0.843 & 0.495 & 0.25 & 0.213 & 0.085
pro pvalue 0.154 & 0.534 & 0.515 & 0.101 & 0.124 & 0.066 & 0.156 & 0.084
####################################################################################################


####################################################################################################
Site G21
####################################################################################################
raw kld 79.111 & inf & 5.335 & inf & 1.084 & 0.198 & 0.27 & 0.104
pro kld 0.053 & 0.321 & 0.27 & 0.273 & 0.153 & 0.118 & 0.233 & 0.104


raw pvalue 0.996 & 0.838 & 0.737 & 0.966 & 0.565 & 

## Audios San miguel

In [43]:
import sys
sys.path.append("/home/david/Documentos/Codes/postgresql/")

from utils_psql import ConnectDB
from rich.table import Table
from rich.console import Console


path_env = '/home/david/Documentos/Codes/postgresql/.env'
db = ConnectDB(path_env)

query1 = """SELECT audios.name, audios.folder, raw_indices.acift, 
            raw_indices.adi, raw_indices.beta, raw_indices.m, raw_indices.np, 
            raw_indices.h, raw_indices.aei, raw_indices.ndsi  
            FROM audios
            JOIN raw_indices
            ON audios.cod_audio = raw_indices.cod_audio
            WHERE audios.folder IN ('G0004', 'G0005', 'G0006', 'G0009','G0010',
            'G0015','G0016','G0017','G0018','G0019','G0032','G0033','G0034','G0035','G0036');
            """

query2 = """SELECT audios.name, audios.folder, processed_indices.acift, 
            processed_indices.adi, processed_indices.beta, processed_indices.m, processed_indices.np, 
            processed_indices.h, processed_indices.aei, processed_indices.ndsi  
            FROM audios
            JOIN processed_indices
            ON audios.cod_audio = processed_indices.cod_audio
            WHERE audios.folder IN ('G0004', 'G0005', 'G0006', 'G0009','G0010',
            'G0015','G0016','G0017','G0018','G0019','G0032','G0033','G0034','G0035','G0036');
            """
        
db.execute(query1)
dfr = db.fetchall()


db.execute(query2)
dfp = db.fetchall()

In [64]:
def plot_table():
      table = Table(title="Resultados")

      table.add_column("Folder1", justify="center", style="#fdfd66")
      table.add_column("Folder2", justify="center", style="#FA8072")
      table.add_column("type", justify="center", style="#FA8072")
      table.add_column("metric", justify="center", style="#FA8072")
      table.add_column("m", justify="center", style="#64b7f6")

      return table


In [66]:
dfr.dropna(inplace=True)
dfp.dropna(inplace=True)

sitios = sorted(list(dfr.folder.unique()))
indices = list(dfr.columns[2:])

for s in sitios:
    table = plot_table()
    for f in sitios:
        raw_a = dfr.query(f"folder == '{s}'")
        raw_s = dfr.query(f"folder == '{f}'")

        pro_a = dfp.query(f"folder == '{s}'")
        pro_s = dfp.query(f"folder == '{f}'")
    
        kld_r = []
        pvalue_r = []

        kld_p = []
        pvalue_p = []


        y = 'm'
        pdf1_r, pdf2_r, p = get_pdf(raw_a[y].values,raw_s[y].values,300)
        mr1 = kl_divergence(pdf1_r,pdf2_r)
        mr3 = d_kolmogorov(pdf1_r,pdf2_r)

        pdf1_p, pdf2_p, x = get_pdf(pro_a[y].values,pro_s[y].values,300)
        mp1 = kl_divergence(pdf1_p,pdf2_p)
        mp3 = d_kolmogorov(pdf1_p,pdf2_p)

        pvalue_p.append(np.round(mp3,3))

        l1 = [s, f , 'raw','kld', str(np.round(mr1,3))] 
        l2 = [s, f , 'raw','dks', str(np.round(mr3,3))]

        l3 = [s, f , 'processed','kld', str(np.round(mp1,3))]

        l4 = [s, f , 'processed','dks', str(np.round(mp3,3))]

        
        table.add_row(l1[0],l1[1],l1[2],l1[3],l1[4])
        table.add_row(l2[0],l2[1],l2[2],l2[3],l2[4])
        table.add_row(l3[0],l3[1],l3[2],l3[3],l3[4])
        table.add_row(l4[0],l4[1],l4[2],l4[3],l4[4])
        

    console = Console()
    console.print(table)

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0004  │  G0004  │    raw    │  kld   │  0.0  │
│  G0004  │  G0004  │    raw    │  dks   │  0.0  │
│  G0004  │  G0004  │ processed │  kld   │  0.0  │
│  G0004  │  G0004  │ processed │  dks   │  0.0  │
│  G0004  │  G0005  │    raw    │  kld   │ 0.076 │
│  G0004  │  G0005  │    raw    │  dks   │ 0.189 │
│  G0004  │  G0005  │ processed │  kld   │ 0.151 │
│  G0004  │  G0005  │ processed │  dks   │ 0.351 │
│  G0004  │  G0006  │    raw    │  kld   │ 0.003 │
│  G0004  │  G0006  │    raw    │  dks   │ 0.03  │
│  G0004  │  G0006  │ processed │  kld   │ 0.374 │
│  G0004  │  G0006  │ processed │  dks   │ 0.463 │
│  G0004  │  G0009  │    raw    │  kld   │ 0.026 │
│  G0004  │  G0009  │    raw    │  dks   │ 0.112 │
│  G0004  │  G0009  │ processed │  kld   │ 0.318 │
│  G0004  │  G0009  │ processed │  dks   │ 0.476 │
│  G0004  │  G0010  │    raw    │  kld   │ 0.016 │
│  G0004  │  G0010  │    raw    │  dks   │ 0.091 │
│  G0004  │  G0010  │ processed │  kld   │ 0.298 │
│  G0004  │  G0010  │ processed │  dks   │ 0.346 │
│  G0004  │  G0015  │    raw    │  kld   │ 0.026 │
│  G0004  │  G0015  │    raw    │  dks   │ 0.069 │
│  G0004  │  G0015  │ processed │  kld   │ 0.375 │
│  G0004  │  G0015  │ processed │  dks   │ 0.507 │
│  G0004  │  G0016  │    raw    │  kld   │ 0.083 │
│  G0004  │  G0016  │    raw    │  dks   │  0.2  │
│  G0004  │  G0016  │ processed │  kld   │ 0.155 │
│  G0004  │  G0016  │ processed │  dks   │ 0.378 │
│  G0004  │  G0017  │    raw    │  kld   │ 0.002 │
│  G0004  │  G0017  │    raw    │  dks   │ 0.021 │
│  G0004  │  G0017  │ processed │  kld   │ 0.155 │
│  G0004  │  G0017  │ processed │  dks   │ 0.236 │
│  G0004  │  G0018  │    raw    │  kld   │ 0.062 │
│  G0004  │  G0018  │    raw    │  dks   │ 0.151 │
│  G0004  │  G0018  │ processed │  kld   │ 0.52  │
│  G0004  │  G0018  │ processed │  dks   │ 0.553 │
│  G0004  │  G0019  │    raw    │  kld   │ 0.004 │
│  G0004  │  G0019  │    raw    │  dks   │ 0.025 │
│  G0004  │  G0019  │ processed │  kld   │ 0.249 │
│  G0004  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0005  │  G0004  │    raw    │  kld   │ 0.076 │
│  G0005  │  G0004  │    raw    │  dks   │ 0.189 │
│  G0005  │  G0004  │ processed │  kld   │ 0.151 │
│  G0005  │  G0004  │ processed │  dks   │ 0.351 │
│  G0005  │  G0005  │    raw    │  kld   │  0.0  │
│  G0005  │  G0005  │    raw    │  dks   │  0.0  │
│  G0005  │  G0005  │ processed │  kld   │  0.0  │
│  G0005  │  G0005  │ processed │  dks   │  0.0  │
│  G0005  │  G0006  │    raw    │  kld   │ 0.051 │
│  G0005  │  G0006  │    raw    │  dks   │ 0.166 │
│  G0005  │  G0006  │ processed │  kld   │ 0.012 │
│  G0005  │  G0006  │ processed │  dks   │ 0.159 │
│  G0005  │  G0009  │    raw    │  kld   │ 0.012 │
│  G0005  │  G0009  │    raw    │  dks   │ 0.08  │
│  G0005  │  G0009  │ processed │  kld   │  0.0  │
│  G0005  │  G0009  │ processed │  dks   │ 0.149 │
│  G0005  │  G0010  │    raw    │  kld   │ 0.024 │
│  G0005  │  G0010  │    raw    │  dks   │ 0.105 │
│  G0005  │  G0010  │ processed │  kld   │ 0.089 │
│  G0005  │  G0010  │ processed │  dks   │ 0.267 │
│  G0005  │  G0015  │    raw    │  kld   │ 0.021 │
│  G0005  │  G0015  │    raw    │  dks   │ 0.127 │
│  G0005  │  G0015  │ processed │  kld   │ 0.047 │
│  G0005  │  G0015  │ processed │  dks   │ 0.276 │
│  G0005  │  G0016  │    raw    │  kld   │ 0.001 │
│  G0005  │  G0016  │    raw    │  dks   │ 0.035 │
│  G0005  │  G0016  │ processed │  kld   │  0.0  │
│  G0005  │  G0016  │ processed │  dks   │ 0.199 │
│  G0005  │  G0017  │    raw    │  kld   │ 0.063 │
│  G0005  │  G0017  │    raw    │  dks   │ 0.181 │
│  G0005  │  G0017  │ processed │  kld   │ 0.033 │
│  G0005  │  G0017  │ processed │  dks   │ 0.253 │
│  G0005  │  G0018  │    raw    │  kld   │ 0.004 │
│  G0005  │  G0018  │    raw    │  dks   │ 0.053 │
│  G0005  │  G0018  │ processed │  kld   │  0.0  │
│  G0005  │  G0018  │ processed │  dks   │ 0.238 │
│  G0005  │  G0019  │    raw    │  kld   │ 0.064 │
│  G0005  │  G0019  │    raw    │  dks   │ 0.189 │
│  G0005  │  G0019  │ processed │  kld   │  0.0  │
│  G0005  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0006  │  G0004  │    raw    │  kld   │ 0.003 │
│  G0006  │  G0004  │    raw    │  dks   │ 0.03  │
│  G0006  │  G0004  │ processed │  kld   │ 0.374 │
│  G0006  │  G0004  │ processed │  dks   │ 0.463 │
│  G0006  │  G0005  │    raw    │  kld   │ 0.051 │
│  G0006  │  G0005  │    raw    │  dks   │ 0.166 │
│  G0006  │  G0005  │ processed │  kld   │ 0.012 │
│  G0006  │  G0005  │ processed │  dks   │ 0.159 │
│  G0006  │  G0006  │    raw    │  kld   │  0.0  │
│  G0006  │  G0006  │    raw    │  dks   │  0.0  │
│  G0006  │  G0006  │ processed │  kld   │  0.0  │
│  G0006  │  G0006  │ processed │  dks   │  0.0  │
│  G0006  │  G0009  │    raw    │  kld   │ 0.013 │
│  G0006  │  G0009  │    raw    │  dks   │ 0.086 │
│  G0006  │  G0009  │ processed │  kld   │ 0.034 │
│  G0006  │  G0009  │ processed │  dks   │ 0.135 │
│  G0006  │  G0010  │    raw    │  kld   │ 0.005 │
│  G0006  │  G0010  │    raw    │  dks   │ 0.062 │
│  G0006  │  G0010  │ processed │  kld   │ 0.158 │
│  G0006  │  G0010  │ processed │  dks   │ 0.374 │
│  G0006  │  G0015  │    raw    │  kld   │ 0.011 │
│  G0006  │  G0015  │    raw    │  dks   │ 0.043 │
│  G0006  │  G0015  │ processed │  kld   │ 0.089 │
│  G0006  │  G0015  │ processed │  dks   │ 0.262 │
│  G0006  │  G0016  │    raw    │  kld   │ 0.057 │
│  G0006  │  G0016  │    raw    │  dks   │ 0.178 │
│  G0006  │  G0016  │ processed │  kld   │ 0.019 │
│  G0006  │  G0016  │ processed │  dks   │ 0.286 │
│  G0006  │  G0017  │    raw    │  kld   │ 0.001 │
│  G0006  │  G0017  │    raw    │  dks   │ 0.015 │
│  G0006  │  G0017  │ processed │  kld   │ 0.201 │
│  G0006  │  G0017  │ processed │  dks   │ 0.405 │
│  G0006  │  G0018  │    raw    │  kld   │ 0.036 │
│  G0006  │  G0018  │    raw    │  dks   │ 0.124 │
│  G0006  │  G0018  │ processed │  kld   │  0.0  │
│  G0006  │  G0018  │ processed │  dks   │ 0.122 │
│  G0006  │  G0019  │    raw    │  kld   │ 0.002 │
│  G0006  │  G0019  │    raw    │  dks   │ 0.024 │
│  G0006  │  G0019  │ processed │  kld   │ 0.015 │
│  G0006  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0009  │  G0004  │    raw    │  kld   │ 0.026 │
│  G0009  │  G0004  │    raw    │  dks   │ 0.112 │
│  G0009  │  G0004  │ processed │  kld   │ 0.318 │
│  G0009  │  G0004  │ processed │  dks   │ 0.476 │
│  G0009  │  G0005  │    raw    │  kld   │ 0.012 │
│  G0009  │  G0005  │    raw    │  dks   │ 0.08  │
│  G0009  │  G0005  │ processed │  kld   │  0.0  │
│  G0009  │  G0005  │ processed │  dks   │ 0.149 │
│  G0009  │  G0006  │    raw    │  kld   │ 0.013 │
│  G0009  │  G0006  │    raw    │  dks   │ 0.086 │
│  G0009  │  G0006  │ processed │  kld   │ 0.034 │
│  G0009  │  G0006  │ processed │  dks   │ 0.135 │
│  G0009  │  G0009  │    raw    │  kld   │  0.0  │
│  G0009  │  G0009  │    raw    │  dks   │  0.0  │
│  G0009  │  G0009  │ processed │  kld   │  0.0  │
│  G0009  │  G0009  │ processed │  dks   │  0.0  │
│  G0009  │  G0010  │    raw    │  kld   │ 0.002 │
│  G0009  │  G0010  │    raw    │  dks   │ 0.041 │
│  G0009  │  G0010  │ processed │  kld   │ 0.112 │
│  G0009  │  G0010  │ processed │  dks   │ 0.375 │
│  G0009  │  G0015  │    raw    │  kld   │ 0.002 │
│  G0009  │  G0015  │    raw    │  dks   │ 0.055 │
│  G0009  │  G0015  │ processed │  kld   │  0.0  │
│  G0009  │  G0015  │ processed │  dks   │ 0.145 │
│  G0009  │  G0016  │    raw    │  kld   │ 0.017 │
│  G0009  │  G0016  │    raw    │  dks   │ 0.094 │
│  G0009  │  G0016  │ processed │  kld   │  0.0  │
│  G0009  │  G0016  │ processed │  dks   │ 0.318 │
│  G0009  │  G0017  │    raw    │  kld   │ 0.018 │
│  G0009  │  G0017  │    raw    │  dks   │ 0.101 │
│  G0009  │  G0017  │ processed │  kld   │ 0.081 │
│  G0009  │  G0017  │ processed │  dks   │ 0.364 │
│  G0009  │  G0018  │    raw    │  kld   │ 0.007 │
│  G0009  │  G0018  │    raw    │  dks   │ 0.043 │
│  G0009  │  G0018  │ processed │  kld   │  0.0  │
│  G0009  │  G0018  │ processed │  dks   │ 0.11  │
│  G0009  │  G0019  │    raw    │  kld   │ 0.02  │
│  G0009  │  G0019  │    raw    │  dks   │ 0.109 │
│  G0009  │  G0019  │ processed │  kld   │  0.0  │
│  G0009  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0010  │  G0004  │    raw    │  kld   │ 0.016 │
│  G0010  │  G0004  │    raw    │  dks   │ 0.091 │
│  G0010  │  G0004  │ processed │  kld   │ 0.298 │
│  G0010  │  G0004  │ processed │  dks   │ 0.346 │
│  G0010  │  G0005  │    raw    │  kld   │ 0.024 │
│  G0010  │  G0005  │    raw    │  dks   │ 0.105 │
│  G0010  │  G0005  │ processed │  kld   │ 0.089 │
│  G0010  │  G0005  │ processed │  dks   │ 0.267 │
│  G0010  │  G0006  │    raw    │  kld   │ 0.005 │
│  G0010  │  G0006  │    raw    │  dks   │ 0.062 │
│  G0010  │  G0006  │ processed │  kld   │ 0.158 │
│  G0010  │  G0006  │ processed │  dks   │ 0.374 │
│  G0010  │  G0009  │    raw    │  kld   │ 0.002 │
│  G0010  │  G0009  │    raw    │  dks   │ 0.041 │
│  G0010  │  G0009  │ processed │  kld   │ 0.112 │
│  G0010  │  G0009  │ processed │  dks   │ 0.375 │
│  G0010  │  G0010  │    raw    │  kld   │  0.0  │
│  G0010  │  G0010  │    raw    │  dks   │  0.0  │
│  G0010  │  G0010  │ processed │  kld   │  0.0  │
│  G0010  │  G0010  │ processed │  dks   │  0.0  │
│  G0010  │  G0015  │    raw    │  kld   │ 0.005 │
│  G0010  │  G0015  │    raw    │  dks   │ 0.051 │
│  G0010  │  G0015  │ processed │  kld   │ 0.329 │
│  G0010  │  G0015  │ processed │  dks   │ 0.519 │
│  G0010  │  G0016  │    raw    │  kld   │ 0.03  │
│  G0010  │  G0016  │    raw    │  dks   │ 0.119 │
│  G0010  │  G0016  │ processed │  kld   │  0.0  │
│  G0010  │  G0016  │ processed │  dks   │ 0.092 │
│  G0010  │  G0017  │    raw    │  kld   │ 0.009 │
│  G0010  │  G0017  │    raw    │  dks   │ 0.077 │
│  G0010  │  G0017  │ processed │  kld   │  0.0  │
│  G0010  │  G0017  │ processed │  dks   │ 0.146 │
│  G0010  │  G0018  │    raw    │  kld   │ 0.014 │
│  G0010  │  G0018  │    raw    │  dks   │ 0.068 │
│  G0010  │  G0018  │ processed │  kld   │ 0.129 │
│  G0010  │  G0018  │ processed │  dks   │ 0.453 │
│  G0010  │  G0019  │    raw    │  kld   │ 0.009 │
│  G0010  │  G0019  │    raw    │  dks   │ 0.084 │
│  G0010  │  G0019  │ processed │  kld   │ 0.17  │
│  G0010  │  G0019  │ processed[38;2;25

...

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0032  │  G0004  │    raw    │  kld   │ 0.063 │
│  G0032  │  G0004  │    raw    │  dks   │ 0.179 │
│  G0032  │  G0004  │ processed │  kld   │ 0.218 │
│  G0032  │  G0004  │ processed │  dks   │ 0.406 │
│  G0032  │  G0005  │    raw    │  kld   │ 0.002 │
│  G0032  │  G0005  │    raw    │  dks   │ 0.044 │
│  G0032  │  G0005  │ processed │  kld   │  0.0  │
│  G0032  │  G0005  │ processed │  dks   │ 0.071 │
│  G0032  │  G0006  │    raw    │  kld   │ 0.037 │
│  G0032  │  G0006  │    raw    │  dks   │ 0.153 │
│  G0032  │  G0006  │ processed │  kld   │ 0.019 │
│  G0032  │  G0006  │ processed │  dks   │ 0.148 │
│  G0032  │  G0009  │    raw    │  kld   │ 0.008 │
│  G0032  │  G0009  │    raw    │  dks   │ 0.068 │
│  G0032  │  G0009  │ processed │  kld   │  0.0  │
│  G0032  │  G0009  │ processed │  dks   │ 0.179 │
│  G0032  │  G0010  │    raw    │  kld   │ 0.016 │
│  G0032  │  G0010  │    raw    │  dks   │ 0.091 │
│  G0032  │  G0010  │ processed │  kld   │ 0.067 │
│  G0032  │  G0010  │ processed │  dks   │ 0.228 │
│  G0032  │  G0015  │    raw    │  kld   │ 0.015 │
│  G0032  │  G0015  │    raw    │  dks   │ 0.111 │
│  G0032  │  G0015  │ processed │  kld   │ 0.079 │
│  G0032  │  G0015  │ processed │  dks   │ 0.311 │
│  G0032  │  G0016  │    raw    │  kld   │ 0.005 │
│  G0032  │  G0016  │    raw    │  dks   │ 0.071 │
│  G0032  │  G0016  │ processed │  kld   │  0.0  │
│  G0032  │  G0016  │ processed │  dks   │ 0.152 │
│  G0032  │  G0017  │    raw    │  kld   │ 0.047 │
│  G0032  │  G0017  │    raw    │  dks   │ 0.168 │
│  G0032  │  G0017  │ processed │  kld   │ 0.076 │
│  G0032  │  G0017  │ processed │  dks   │ 0.284 │
│  G0032  │  G0018  │    raw    │  kld   │ 0.001 │
│  G0032  │  G0018  │    raw    │  dks   │ 0.039 │
│  G0032  │  G0018  │ processed │  kld   │  0.0  │
│  G0032  │  G0018  │ processed │  dks   │ 0.227 │
│  G0032  │  G0019  │    raw    │  kld   │ 0.049 │
│  G0032  │  G0019  │    raw    │  dks   │ 0.175 │
│  G0032  │  G0019  │ processed │  kld   │  0.0  │
│  G0032  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0033  │  G0004  │    raw    │  kld   │ 0.116 │
│  G0033  │  G0004  │    raw    │  dks   │ 0.23  │
│  G0033  │  G0004  │ processed │  kld   │ 0.329 │
│  G0033  │  G0004  │ processed │  dks   │ 0.457 │
│  G0033  │  G0005  │    raw    │  kld   │ 0.004 │
│  G0033  │  G0005  │    raw    │  dks   │ 0.041 │
│  G0033  │  G0005  │ processed │  kld   │  0.0  │
│  G0033  │  G0005  │ processed │  dks   │ 0.282 │
│  G0033  │  G0006  │    raw    │  kld   │ 0.081 │
│  G0033  │  G0006  │    raw    │  dks   │ 0.207 │
│  G0033  │  G0006  │ processed │  kld   │ 0.142 │
│  G0033  │  G0006  │ processed │  dks   │ 0.272 │
│  G0033  │  G0009  │    raw    │  kld   │ 0.031 │
│  G0033  │  G0009  │    raw    │  dks   │ 0.121 │
│  G0033  │  G0009  │ processed │  kld   │  0.0  │
│  G0033  │  G0009  │ processed │  dks   │ 0.149 │
│  G0033  │  G0010  │    raw    │  kld   │ 0.048 │
│  G0033  │  G0010  │    raw    │  dks   │ 0.146 │
│  G0033  │  G0010  │ processed │  kld   │ 0.207 │
│  G0033  │  G0010  │ processed │  dks   │ 0.453 │
│  G0033  │  G0015  │    raw    │  kld   │ 0.043 │
│  G0033  │  G0015  │    raw    │  dks   │ 0.166 │
│  G0033  │  G0015  │ processed │  kld   │  0.0  │
│  G0033  │  G0015  │ processed │  dks   │ 0.106 │
│  G0033  │  G0016  │    raw    │  kld   │ 0.005 │
│  G0033  │  G0016  │    raw    │  dks   │ 0.035 │
│  G0033  │  G0016  │ processed │  kld   │ 0.18  │
│  G0033  │  G0016  │ processed │  dks   │ 0.414 │
│  G0033  │  G0017  │    raw    │  kld   │ 0.099 │
│  G0033  │  G0017  │    raw    │  dks   │ 0.222 │
│  G0033  │  G0017  │ processed │  kld   │ 0.131 │
│  G0033  │  G0017  │ processed │  dks   │ 0.397 │
│  G0033  │  G0018  │    raw    │  kld   │ 0.012 │
│  G0033  │  G0018  │    raw    │  dks   │ 0.088 │
│  G0033  │  G0018  │ processed │  kld   │ 0.083 │
│  G0033  │  G0018  │ processed │  dks   │ 0.214 │
│  G0033  │  G0019  │    raw    │  kld   │ 0.099 │
│  G0033  │  G0019  │    raw    │  dks   │ 0.229 │
│  G0033  │  G0019  │ processed │  kld   │  0.0  │
│  G0033  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0034  │  G0004  │    raw    │  kld   │ 0.003 │
│  G0034  │  G0004  │    raw    │  dks   │ 0.049 │
│  G0034  │  G0004  │ processed │  kld   │ 0.14  │
│  G0034  │  G0004  │ processed │  dks   │ 0.251 │
│  G0034  │  G0005  │    raw    │  kld   │ 0.046 │
│  G0034  │  G0005  │    raw    │  dks   │ 0.141 │
│  G0034  │  G0005  │ processed │  kld   │ 0.014 │
│  G0034  │  G0005  │ processed │  dks   │ 0.238 │
│  G0034  │  G0006  │    raw    │  kld   │ 0.001 │
│  G0034  │  G0006  │    raw    │  dks   │ 0.04  │
│  G0034  │  G0006  │ processed │  kld   │ 0.186 │
│  G0034  │  G0006  │ processed │  dks   │ 0.38  │
│  G0034  │  G0009  │    raw    │  kld   │ 0.011 │
│  G0034  │  G0009  │    raw    │  dks   │ 0.067 │
│  G0034  │  G0009  │ processed │  kld   │ 0.045 │
│  G0034  │  G0009  │ processed │  dks   │ 0.342 │
│  G0034  │  G0010  │    raw    │  kld   │ 0.004 │
│  G0034  │  G0010  │    raw    │  dks   │ 0.042 │
│  G0034  │  G0010  │ processed │  kld   │ 0.048 │
│  G0034  │  G0010  │ processed │  dks   │ 0.113 │
│  G0034  │  G0015  │    raw    │  kld   │ 0.012 │
│  G0034  │  G0015  │    raw    │  dks   │ 0.053 │
│  G0034  │  G0015  │ processed │  kld   │ 0.243 │
│  G0034  │  G0015  │ processed │  dks   │ 0.481 │
│  G0034  │  G0016  │    raw    │  kld   │ 0.051 │
│  G0034  │  G0016  │    raw    │  dks   │ 0.152 │
│  G0034  │  G0016  │ processed │  kld   │  0.0  │
│  G0034  │  G0016  │ processed │  dks   │ 0.148 │
│  G0034  │  G0017  │    raw    │  kld   │ 0.002 │
│  G0034  │  G0017  │    raw    │  dks   │ 0.045 │
│  G0034  │  G0017  │ processed │  kld   │ 0.008 │
│  G0034  │  G0017  │ processed │  dks   │ 0.038 │
│  G0034  │  G0018  │    raw    │  kld   │ 0.038 │
│  G0034  │  G0018  │    raw    │  dks   │ 0.101 │
│  G0034  │  G0018  │ processed │  kld   │ 0.124 │
│  G0034  │  G0018  │ processed │  dks   │ 0.447 │
│  G0034  │  G0019  │    raw    │  kld   │ 0.005 │
│  G0034  │  G0019  │    raw    │  dks   │ 0.054 │
│  G0034  │  G0019  │ processed │  kld   │ 0.094 │
│  G0034  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0035  │  G0004  │    raw    │  kld   │ 0.097 │
│  G0035  │  G0004  │    raw    │  dks   │ 0.213 │
│  G0035  │  G0004  │ processed │  kld   │ 0.131 │
│  G0035  │  G0004  │ processed │  dks   │ 0.335 │
│  G0035  │  G0005  │    raw    │  kld   │ 0.004 │
│  G0035  │  G0005  │    raw    │  dks   │ 0.044 │
│  G0035  │  G0005  │ processed │  kld   │  0.0  │
│  G0035  │  G0005  │ processed │  dks   │ 0.144 │
│  G0035  │  G0006  │    raw    │  kld   │ 0.063 │
│  G0035  │  G0006  │    raw    │  dks   │ 0.187 │
│  G0035  │  G0006  │ processed │  kld   │ 0.024 │
│  G0035  │  G0006  │ processed │  dks   │ 0.26  │
│  G0035  │  G0009  │    raw    │  kld   │ 0.022 │
│  G0035  │  G0009  │    raw    │  dks   │ 0.102 │
│  G0035  │  G0009  │ processed │  kld   │  0.0  │
│  G0035  │  G0009  │ processed │  dks   │ 0.254 │
│  G0035  │  G0010  │    raw    │  kld   │ 0.034 │
│  G0035  │  G0010  │    raw    │  dks   │ 0.125 │
│  G0035  │  G0010  │ processed │  kld   │ 0.019 │
│  G0035  │  G0010  │ processed │  dks   │ 0.123 │
│  G0035  │  G0015  │    raw    │  kld   │ 0.032 │
│  G0035  │  G0015  │    raw    │  dks   │ 0.145 │
│  G0035  │  G0015  │ processed │  kld   │ 0.141 │
│  G0035  │  G0015  │ processed │  dks   │ 0.397 │
│  G0035  │  G0016  │    raw    │  kld   │ 0.008 │
│  G0035  │  G0016  │    raw    │  dks   │ 0.052 │
│  G0035  │  G0016  │ processed │  kld   │  0.0  │
│  G0035  │  G0016  │ processed │  dks   │ 0.066 │
│  G0035  │  G0017  │    raw    │  kld   │ 0.078 │
│  G0035  │  G0017  │    raw    │  dks   │ 0.202 │
│  G0035  │  G0017  │ processed │  kld   │ 0.016 │
│  G0035  │  G0017  │ processed │  dks   │ 0.182 │
│  G0035  │  G0018  │    raw    │  kld   │ 0.005 │
│  G0035  │  G0018  │    raw    │  dks   │ 0.063 │
│  G0035  │  G0018  │ processed │  kld   │  0.0  │
│  G0035  │  G0018  │ processed │  dks   │ 0.332 │
│  G0035  │  G0019  │    raw    │  kld   │ 0.076 │
│  G0035  │  G0019  │    raw    │  dks   │ 0.208 │
│  G0035  │  G0019  │ processed │  kld   │  0.0  │
│  G0035  │  G0019  │ processed[38;2;25

                    Resultados                    
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Folder1 ┃ Folder2 ┃   type    ┃ metric ┃   m   ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│  G0036  │  G0004  │    raw    │  kld   │ 0.044 │
│  G0036  │  G0004  │    raw    │  dks   │ 0.15  │
│  G0036  │  G0004  │ processed │  kld   │ 0.418 │
│  G0036  │  G0004  │ processed │  dks   │ 0.418 │
│  G0036  │  G0005  │    raw    │  kld   │ 0.004 │
│  G0036  │  G0005  │    raw    │  dks   │ 0.043 │
│  G0036  │  G0005  │ processed │  kld   │ 0.12  │
│  G0036  │  G0005  │ processed │  dks   │ 0.227 │
│  G0036  │  G0006  │    raw    │  kld   │ 0.024 │
│  G0036  │  G0006  │    raw    │  dks   │ 0.124 │
│  G0036  │  G0006  │ processed │  kld   │ 0.152 │
│  G0036  │  G0006  │ processed │  dks   │ 0.317 │
│  G0036  │  G0009  │    raw    │  kld   │ 0.002 │
│  G0036  │  G0009  │    raw    │  dks   │ 0.039 │
│  G0036  │  G0009  │ processed │  kld   │ 0.137 │
│  G0036  │  G0009  │ processed │  dks   │ 0.342 │
│  G0036  │  G0010  │    raw    │  kld   │ 0.008 │
│  G0036  │  G0010  │    raw    │  dks   │ 0.075 │
│  G0036  │  G0010  │ processed │  kld   │  0.0  │
│  G0036  │  G0010  │ processed │  dks   │ 0.087 │
│  G0036  │  G0015  │    raw    │  kld   │ 0.008 │
│  G0036  │  G0015  │    raw    │  dks   │ 0.084 │
│  G0036  │  G0015  │ processed │  kld   │ 0.34  │
│  G0036  │  G0015  │ processed │  dks   │ 0.484 │
│  G0036  │  G0016  │    raw    │  kld   │ 0.008 │
│  G0036  │  G0016  │    raw    │  dks   │ 0.064 │
│  G0036  │  G0016  │ processed │  kld   │  0.0  │
│  G0036  │  G0016  │ processed │  dks   │ 0.049 │
│  G0036  │  G0017  │    raw    │  kld   │ 0.034 │
│  G0036  │  G0017  │    raw    │  dks   │ 0.139 │
│  G0036  │  G0017  │ processed │  kld   │  0.0  │
│  G0036  │  G0017  │ processed │  dks   │ 0.231 │
│  G0036  │  G0018  │    raw    │  kld   │ 0.002 │
│  G0036  │  G0018  │    raw    │  dks   │ 0.034 │
│  G0036  │  G0018  │ processed │  kld   │ 0.133 │
│  G0036  │  G0018  │ processed │  dks   │ 0.408 │
│  G0036  │  G0019  │    raw    │  kld   │ 0.034 │
│  G0036  │  G0019  │    raw    │  dks   │ 0.146 │
│  G0036  │  G0019  │ processed │  kld   │ 0.177 │
│  G0036  │  G0019  │ processed[38;2;25